In [4]:
import os, csv, pickle, json, pymongo, copy, re, time, datetime, math
import pandas as pd
from pymongo import MongoClient

In [5]:
pd.set_option('display.max_columns', 50)

In [6]:
jcr=pd.read_csv("../JCR/jcr2023.tsv",sep="\t")

In [7]:
len(jcr)

20522

In [8]:
jcr[-10:-1]

,rank,journalTitle,abbrJournal,year,edition,issn,totalCites,journalImpactFactor,impactFactorWithoutJournalSelfCites,fiveYearImpactFactor,immediacyIndex,citableItems,citedHalfLife,citingHalfLife,eigenFactorScore,articleInfluenceScore,citesCurrent,articlesCurrent,selfCites,impactFactorSelfCites,originalResearch,jifPercentile,normEigenFactor
20512,9855,European Journal of Orthopaedic Surgery and Tr...,EUR J ORTHOP SURG TR,2023,NaN,1633-8065,3574,1.7,1.6,NaN,0.3,326,4.7,8.3,0.00518,NaN,0,0,0,0,0,NaN,1.11597
20513,9855,European Journal of Transport and Infrastructu...,EUR J TRANSP INFRAST,2023,NaN,1567-7133,664,1.7,1.7,1.5,0.5,19,8.4,6.2,0.00036,0.336,0,0,0,0,0,9.5,0.07776
20514,9855,European Journal on Criminal Policy and Research,EUR J CRIM POLICY RE,2023,NaN,0928-1371,773,1.7,1.6,1.9,0.5,35,6.8,8.0,0.00092,0.630,0,0,0,0,0,44.2,0.19839
20515,9855,EXPERIMENTAL LUNG RESEARCH,EXP LUNG RES,2023,NaN,0190-2148,1340,1.7,1.7,2.0,0.3,24,9.5,6.0,0.00067,0.379,0,0,0,0,0,8.5,0.14451
20516,9855,FEDERAL RESERVE BANK OF ST LOUIS REVIEW,FED RESERVE BANK ST,2023,NaN,0014-9187,576,1.7,1.6,1.4,0.4,18,13.5,6.7,0.00069,0.857,0,0,0,0,0,40.1,0.14960
20517,9855,Feminist Media Studies,FEM MEDIA STUD,2023,NaN,1468-0777,2273,1.7,1.4,2.3,0.3,175,4.9,8.2,0.00502,1.025,0,0,0,0,0,55.7,1.08187
20518,9855,FIBER AND INTEGRATED OPTICS,FIBER INTEGRATED OPT,2023,NaN,0146-8030,265,1.7,1.7,1.0,0.3,8,8.8,6.4,0.00013,0.161,0,0,0,0,0,27.5,0.02988
20519,9855,Field Methods,FIELD METHOD,2023,NaN,1525-822X,3624,1.7,1.6,3.9,0.3,41,16.3,9.4,0.00150,1.336,0,0,0,0,0,67.6,0.32450
20520,9855,FINANCE AND STOCHASTICS,FINANC STOCH,2023,NaN,0949-2984,1384,1.7,1.6,2.3,0.3,31,12.7,13.2,0.00202,1.563,0,0,0,0,0,59.6,0.43612


In [9]:
jt2if_all={}
for i in range(len(jcr)):
    if jcr.iloc[i,7]=='<0.1':
        jcr.iloc[i,7]='0.0'
    if math.isnan(float(jcr.iloc[i,7])):
        jt2if_all[jcr.iloc[i,1].lower()]=0.0
    else:
        jt2if_all[jcr.iloc[i,1].lower()]=float(jcr.iloc[i,7])
jt2if_all['aging']=jt2if_all.get('aging-us')

In [10]:
jt2if_brief={}
for i in range(len(jcr)):
    if jcr.iloc[i,7]=='<0.1':
        jcr.iloc[i,7]='0.0'
    if math.isnan(float(jcr.iloc[i,7])):
        jt2if_brief[jcr.iloc[i,1].lower()]=0.0
    else:
        jt2if_brief[jcr.iloc[i,2].lower()]=float(jcr.iloc[i,7])
jt2if_brief['aging']=jt2if_brief.get('aging-us')

In [11]:
jt2if5_all={}
for i in range(len(jcr)):
    if jcr.iloc[i,9]=='<0.1':
        jcr.iloc[i,9]='0.0'
    if math.isnan(float(jcr.iloc[i,9])):
        jt2if5_all[jcr.iloc[i,1].lower()]=0.0
    else:
        jt2if5_all[jcr.iloc[i,1].lower()]=float(jcr.iloc[i,9])
jt2if5_all['aging']=jt2if5_all.get('aging-us')

In [12]:
jt2if5_brief={}
for i in range(len(jcr)):
    if jcr.iloc[i,9]=='<0.1':
        jcr.iloc[i,9]='0.0'
    if math.isnan(float(jcr.iloc[i,9])):
        jt2if5_brief[jcr.iloc[i,1].lower()]=0.0
    else:
        jt2if5_brief[jcr.iloc[i,2].lower()]=float(jcr.iloc[i,9])
jt2if5_brief['aging']=jt2if5_brief.get('aging-us')

In [15]:
list(jt2if5_brief.items())[:2]

[('ca-cancer j clin', '307.2'), ('lancet', '118.1')]

In [16]:
client = MongoClient(username="hald",password="mclab236")

In [17]:
db = client.hald

In [18]:
collections_pubtator_info = db.pubtator_info

In [19]:
pubtator_info=list(collections_pubtator_info.find())

In [20]:
len(pubtator_info)

208697

In [21]:
pubtator_info[10]

{'_id': ObjectId('6393478a5303aaecac2745e7'),
 'PMID': '35794516',
 'OWN': 'NLM',
 'STAT': 'MEDLINE',
 'DCOM': '20220708',
 'LR': '20220708',
 'IS': '1471-227X (Electronic) 1471-227X (Linking)',
 'VI': '22',
 'IP': '1',
 'DP': '2022 Jul 6',
 'TI': 'Long-term prognosis and clinical course of choking-induced cardiac arrest in patients without the return of spontaneous circulation at hospital arrival: a population-based community study from the Shizuoka Kokuho Database.',
 'PG': '120',
 'LID': '10.1186/s12873-022-00676-8 [doi]',
 'AB': 'BACKGROUND: The risk of choking increases with aging, and the number of cases of choking-induced cardiac arrest is increasing. However, few studies have examined the prognosis of choking-induced cardiac arrest. The aim of this study was to reveal the rates of survival and dependence on devices in the long term after choking-induced cardiac arrest. METHODS: We analyzed data from the Shizuoka Kokuho Database, which consists of claims data of approximately 2.

In [22]:
pubtator_info_dict={}
for i in pubtator_info:
    pubtator_info_dict[i.get('PMID')]=i

In [23]:
collections_Relations_final=db.Relations_final

In [24]:
triplelist=list(collections_Relations_final.find())

In [25]:
triplelist[0]

{'_id': ObjectId('64611763a2e9cde11765b7be'),
 'PMID': '30781849',
 'entity1': 'Pulmonary Disease, Chronic Obstructive',
 'entity2': 'Inflammation',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tob

In [26]:
len(triplelist)

150664

In [27]:
triplelist[0]

{'_id': ObjectId('64611763a2e9cde11765b7be'),
 'PMID': '30781849',
 'entity1': 'Pulmonary Disease, Chronic Obstructive',
 'entity2': 'Inflammation',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tob

In [28]:
be=['am','is','are','was','were','been','have been','has been']
bedict={}
for i in be:
    bedict[i]='be'

triplelist_takeoff=[]
transformed_triples=set()
box=[]
box2=[]
acount=0
for i in triplelist:
    maxlength=[]
    allennlp_list=[]
    if i.get('openie_final'):
        maxlength=i.get('openie_final')[0]
        for j in i.get('openie_final'):
            if len(j[1])>len(maxlength[1]):
                maxlength=j
        mid_list=maxlength[1].split(' ')
        for p,q in enumerate(mid_list):
            if bedict.get(q):
                mid_list[p]='be'
        maxlength[1]=' '.join(mid_list)
        for k in maxlength[1]:
            if k in maxlength[0].split(' ') or k in maxlength[2].split(' '):
                break
        else:
            mid=tuple([maxlength[0],maxlength[1].lower(),maxlength[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid}
                triplelist_takeoff.append(copydict)
                transformed_triples.add(mid)
            if maxlength[1]=='associated' or maxlength[1]=='associate' :
                mid=tuple([maxlength[2],maxlength[1].lower(),maxlength[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)
        for j in i.get('openie_final'):
            if j==maxlength:
                continue
            judge=True
            for k in j[1].split(' '):
                if k not in maxlength[1].split(' '):
                    judge=False
                    break
            if not judge:
                mid_list=j[1].split(' ')
                for p,q in enumerate(mid_list):
                    if bedict.get(q):
                        mid_list[p]='be'
                j[1]=' '.join(mid_list)
                mid=tuple([j[0],j[1].lower(),j[2],i.get('PMID')])
                for k in j[1]:
                    if k in j[0].split(" ") or k in j[1].split(" "):
                        break
                else:
                    if mid not in transformed_triples:
                        copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                  'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                                  'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                                  'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                                    'transformed_triple':mid}
                        triplelist_takeoff.append(copydict)
                        transformed_triples.add(mid)
                    if j[1]=='associated' or j[1]=='associate' :
                        mid=tuple([j[2],j[1].lower(),j[0],i.get('PMID')])
                        if mid not in transformed_triples:
                            copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                      'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                                      'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                                      'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                        'transformed_triple':mid}
                            triplelist_takeoff.append(copydict)
                            transformed_triples.add(mid)
    if i.get('allennlp_final'):
        for j in i.get('allennlp_final'):
            if len(maxlength)>0 and j[1] in maxlength[1].split(' '):
                pass
            else:
                mid=tuple([j[0],j[1].lower(),j[2],i.get('PMID')])
                if j[1] in j[0].split(' ') or j[1] in j[2].split(' '):
                    continue
                if mid not in transformed_triples:
                    allennlp_list.append((j[0],j[2]))
                    copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                              'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                              'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)
                else:
                    box2.append(i)
                if j[1]=='associated' or j[1]=='associate' :
                    mid=tuple([j[2],j[1].lower(),j[0],i.get('PMID')])
                    if mid not in transformed_triples:
                        allennlp_list.append([j[2],j[0]])
                        copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                  'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                                  'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                                  'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                    'transformed_triple':mid}
                        triplelist_takeoff.append(copydict)
                        transformed_triples.add(mid)
                    else:
                        box2.append(i)
    if i.get('networkx'):
        if len(maxlength)>0 and i.get('networkx')[1] in maxlength[1].split(' '):
            box.append(i)
            pass
        else:
            if i.get('networkx')[1] in i.get('networkx')[0].split(' ') or i.get('networkx')[1] in i.get('networkx')[2].split(' '):
                acount+=1
                continue
            mid=tuple([i.get('networkx')[0],i.get('networkx')[1].lower(),i.get('networkx')[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid}
                triplelist_takeoff.append(copydict)
                transformed_triples.add(mid)
            if i.get('networkx')[1]=='associated' or i.get('networkx')[1]=='associate' :
                mid=tuple([i.get('networkx')[2],i.get('networkx')[1].lower(),i.get('networkx')[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)

In [29]:
triplelist_takeoffed=[]
transformed_triples=set()
box=[]
box2=[]
acount=0
for i in triplelist:
    maxlength=[]
    allennlp_list=[]
    if i.get('openie_final'):
        maxlength=i.get('openie_final')[0]
        for j in i.get('openie_final'):
            if len(j[1])>len(maxlength[1]):
                maxlength=j
        mid_list=maxlength[1].split(' ')
        for p,q in enumerate(mid_list):
            if bedict.get(q):
                mid_list[p]='be'
        maxlength[1]=' '.join(mid_list)
        for k in maxlength[1]:
            if k in maxlength[0].split(' ') or k in maxlength[2].split(' '):
                break
        else:
            mid=tuple([maxlength[0],maxlength[1].lower(),maxlength[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid,'method':["open domain"]}
                triplelist_takeoffed.append(copydict)
                transformed_triples.add(mid)
            if maxlength[1]=='associated' or maxlength[1]=='associate' :
                mid=tuple([maxlength[2],maxlength[1].lower(),maxlength[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid,'method':["open domain"]}
                    triplelist_takeoffed.append(copydict)
                    transformed_triples.add(mid)
        for j in i.get('openie_final'):
            if j==maxlength:
                continue
            judge=True
            for k in j[1].split(' '):
                if k not in maxlength[1].split(' '):
                    judge=False
                    break
            if not judge:
                mid_list=j[1].split(' ')
                for p,q in enumerate(mid_list):
                    if bedict.get(q):
                        mid_list[p]='be'
                j[1]=' '.join(mid_list)
                mid=tuple([j[0],j[1].lower(),j[2],i.get('PMID')])
                for k in j[1]:
                    if k in j[0].split(" ") or k in j[1].split(" "):
                        break
                else:
                    if mid not in transformed_triples:
                        copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                  'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                                  'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                                  'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                                    'transformed_triple':mid,'method':["open domain"]}
                        triplelist_takeoffed.append(copydict)
                        transformed_triples.add(mid)
                    if j[1]=='associated' or j[1]=='associate' :
                        mid=tuple([j[2],j[1].lower(),j[0],i.get('PMID')])
                        if mid not in transformed_triples:
                            copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                      'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                                      'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                                      'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                        'transformed_triple':mid,'method':["open domain"]}
                            triplelist_takeoffed.append(copydict)
                            transformed_triples.add(mid)
    if i.get('allennlp_final'):
        for j in i.get('allennlp_final'):
            if len(maxlength)>0 and j[1] in maxlength[1].split(' '):
                pass
            else:
                mid=tuple([j[0],j[1].lower(),j[2],i.get('PMID')])
                if j[1] in j[0].split(' ') or j[1] in j[2].split(' '):
                    continue
                allennlp_list.append((j[0],j[2]))
                copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid,'method':["deep learning"]}
                triplelist_takeoffed.append(copydict)
                transformed_triples.add(mid)
                if j[1]=='associated' or j[1]=='associate' :
                    mid=tuple([j[2],j[1].lower(),j[0],i.get('PMID')])
                    allennlp_list.append([j[2],j[0]])
                    copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid,'method':["deep learning"]}
                    triplelist_takeoffed.append(copydict)
                    transformed_triples.add(mid)
    if i.get('networkx'):
        if len(maxlength)>0 and i.get('networkx')[1] in maxlength[1].split(' '):
            box.append(i)
            pass
        else:
            if i.get('networkx')[1] in i.get('networkx')[0].split(' ') or i.get('networkx')[1] in i.get('networkx')[2].split(' '):
                acount+=1
                continue
            mid=tuple([i.get('networkx')[0],i.get('networkx')[1].lower(),i.get('networkx')[2],i.get('PMID')])
            copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                      'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                      'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                      'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                        'transformed_triple':mid,'method':["shortest path"]}
            triplelist_takeoffed.append(copydict)
            transformed_triples.add(mid)
            if i.get('networkx')[1]=='associated' or i.get('networkx')[1]=='associate' :
                mid=tuple([i.get('networkx')[2],i.get('networkx')[1].lower(),i.get('networkx')[0],i.get('PMID')])

                copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                          'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                          'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                            'transformed_triple':mid,'method':["shortest path"]}
                triplelist_takeoffed.append(copydict)
                transformed_triples.add(mid)

for i in triplelist:
    if i.get('networkx') and i.get('openie_final'):
        print(i)
        break

In [30]:
len(triplelist_takeoffed)

202542

In [31]:
triplelist_takeoffed_dict={}
triplelist_takeoff=[]
for i in triplelist_takeoffed:
    if not triplelist_takeoffed_dict.get((i.get('transformed_triple'),tuple(i.get('triple')),i.get('PMID'),i.get('sentence'),i.get('AB'),i.get('entity_one_type'),i.get('entity_two_type'))):
        triplelist_takeoffed_dict[(i.get('transformed_triple'),tuple(i.get('triple')),i.get('PMID'),i.get('sentence'),i.get('AB'),i.get('entity_one_type'),i.get('entity_two_type'))]=i
    else:
        if i.get('method')[0] in triplelist_takeoffed_dict.get((i.get('transformed_triple'),tuple(i.get('triple')),i.get('PMID'),i.get('sentence'),i.get('AB'),i.get('entity_one_type'),i.get('entity_two_type'))).get('method'):
            continue
        else:
            triplelist_takeoffed_dict[(i.get('transformed_triple'),tuple(i.get('triple')),i.get('PMID'),i.get('sentence'),i.get('AB'),i.get('entity_one_type'),i.get('entity_two_type'))]['method']=triplelist_takeoffed_dict.get((i.get('transformed_triple'),tuple(i.get('triple')),i.get('PMID'),i.get('sentence'),i.get('AB'),i.get('entity_one_type'),i.get('entity_two_type'))).get('method')+i.get('method')

In [32]:
triplelist_takeoff=list(triplelist_takeoffed_dict.values())

In [33]:
for i in triplelist_takeoff:
    if len(i.get('method'))>1:
        print(i)
        break

{'triple': ['COPD', 'defined', 'inflammatory disorder'], 'PMID': '30781849', 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.', 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of th

In [34]:
len(box)

1452

In [35]:
len(box2)

0

In [36]:
print(len(triplelist_takeoff))
print(len(transformed_triples))

175830
171992


In [37]:
triplelist_takeoff[0]

{'triple': ['Chronic obstructive pulmonary disease',
  'defined',
  'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obta

In [38]:
triplelist_takeoff_dict={}
for i in triplelist_takeoff:
    triplelist_takeoff_dict[(i.get('entity_one_transformed'),i.get('triple')[1],i.get('entity_two_transformed'),i.get('PMID'))]=triplelist_takeoff_dict.get((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')),[])+[i]

In [39]:
len(triplelist_takeoff_dict)

158128

In [40]:
triplelist_takeoffs=[]
a=[]
b=[]
c=[]
d=[]
for i,j in triplelist_takeoff_dict.items():
    if len(j)==1:
        triplelist_takeoffs.append(j[0])
    else:
        deprecated_set=set()
        for k in j:
            if k.get('entity_one').lower()==k.get('entity_one_transformed').lower() and k.get('entity_two').lower()==k.get('entity_two_transformed').lower():
                triplelist_takeoffs.append(k)
                a.append(k)
                break
        else:
            for k in j:
                if k.get('entity_one').lower()==k.get('entity_one_transformed').lower():
                    triplelist_takeoffs.append(k)
                    b.append(k)
                    break
            else:
                for k in j:
                    if k.get('entity_two').lower()==k.get('entity_two_transformed').lower():
                        c.append(k)
                        triplelist_takeoffs.append(k)
                        break
                else:
                    d.append(k)
                    triplelist_takeoffs.append(j[0])

In [41]:
names=locals()
for i in ['a','b','c','d']:
    print(len(names[i]))

0
0
0
0


In [42]:
len(triplelist_takeoffs)

158128

In [43]:
len(triplelist_takeoff)

175830

In [44]:
triplelist_takesoff=[]
for i in triplelist_takeoffs:
    if not re.search(r'\`|\~|\!|\@|\#|\$|\%|\^|\&|\*|\(|\)|\_|\+|\=|\{|\[|\}|\]|\;|\:|\'|\"|\,|\<|\.|\>|\/|\?',i.get('triple')[1]):
        if not re.search(r'(\s\d+\s)|(\b\d+\b)',i.get('triple')[1]): 
            triplelist_takesoff.append(i)

In [45]:
triplelistnewcopy=copy.deepcopy(triplelist_takesoff)

In [46]:
deprecated_relations=pd.read_csv('../classofrelations/deprecated.csv',header=None,names=['word'])

In [47]:
len(deprecated_relations)

209

In [48]:
deprecated_relations.head()

,word
0,% for
1,'s III be
2,)evaluate
3,#NAME?
4,#NAME?


In [49]:
deprecated_relations_dict={}
for i in deprecated_relations['word'].to_list():
    deprecated_relations_dict[i.lower()]=True

In [50]:
deprecated_relations_dict.get('have')

True

In [51]:
list(deprecated_relations_dict.items())[20]

('aforemention', True)

In [52]:
triplelistnewcopy[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [53]:
i

'promising'

In [54]:
bedict

{'am': 'be',
 'is': 'be',
 'are': 'be',
 'was': 'be',
 'were': 'be',
 'been': 'be',
 'have been': 'be',
 'has been': 'be'}

In [55]:
not_relations=pd.read_csv('../classofrelations/not_relations.csv',header=None,names=['word'])

In [56]:
len(not_relations)

61

In [57]:
not_relations.head()

,word
0,1p13 near
1,a of
2,abstractdiagnosing
3,acetylcholinesterase
4,apolipoprotein


In [58]:
not_relations_dict={}
for i in not_relations['word'].to_list():
    not_relations_dict[i.lower()]=True

In [59]:
list(not_relations_dict.items())[20]

('near 7q11 be', True)

In [60]:
triplelistnew=[]
for i in triplelistnewcopy: 
    if not deprecated_relations_dict.get(i.get('triple')[1].lower()) and not not_relations_dict.get(i.get('triple')[1].lower()):
        if i.get('triple')[1][0]=='\-':
            continue
        temp_triple=i.get('triple')[1].split(' ')
        for j,k in enumerate(temp_triple):
            if bedict.get(k.lower()):
                temp_triple[j]=bedict.get(k.lower())
        i['triple'][1]=' '.join(temp_triple)
        if i.get('triple')[1].lower()=='be' or bedict.get(i.get('triple')[1].lower()):
            continue
        if i.get('triple')[1].lower()=='means':
            i['triple'][1]='mean'
        if i.get('triple')[1].lower()=='meets':
            i['triple'][1]='meet'
        mid_judge=True
        for j in i.get('triple')[1].split(' '):
            if j.lower() not in i.get('triple')[0].lower().split(' ') and j.lower() not in i.get('triple')[2].lower().split(' '):
                pass
            else:
                mid_judge=False
        if mid_judge:
            triplelistnew.append(i)

In [61]:
len(triplelistnew)

146384

In [62]:
triplelistnew[10]

{'triple': ['Sarcopenia', 'associated', 'constipation'],
 'PMID': '34769606',
 'sentence': '(4) Conclusions: Sarcopenia and slow gait speed associated with constipation in community-dwelling older adults.',
 'AB': '(1) Background: As the clinical relevance of constipation and sarcopenia is not well studied, we aimed to investigate the association between them in older adults. (2) Methods: A cross-sectional study was conducted on 1278 community-dwelling older adults in South Korea. The Rome IV criteria were used to identify patients with clinically defined constipation, while sarcopenia was defined by the Asian Working Group for Sarcopenia consensus. The cohort was classified into three groups: no constipation, self-reported constipation only, and clinically defined constipation. (3) Results: The presence of constipation was associated with sarcopenia and slow gait speed (p < 0.001). After adjustment for possible covariates, the association with sarcopenia attenuated, while that for slo

In [63]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [64]:
entity_set=set()
for i in triplelistnew:
    entity_set.add(i.get('entity_one_transformed'))
    entity_set.add(i.get('entity_two_transformed'))

In [65]:
len(entity_set)

6922

In [66]:
entitydict={}
for i,j in enumerate(triplelist):
    if j.get('entity1') in entity_set and j.get('entity2') in entity_set:
        if not entitydict.get(j.get('entity1')) and not entitydict.get(j.get('entity2')):
            entitydict[j.get('entity1')]=j.get('target1_type')
            entitydict[j.get('entity2')]=j.get('target2_type')
        else:
            if entitydict.get(j.get('entity1')):
                if re.search(j.get('target1_type'), entitydict.get(j.get('entity1'))) :
                    pass
                else:
                    entitydict[j.get('entity1')]+=";"+j.get('target1_type')
            else:
                entitydict[j.get('entity1')]=j.get('target1_type')
            if entitydict.get(j.get('entity2')):
                if re.search(j.get('target2_type'), entitydict.get(j.get('entity2'))) :
                    pass
                else:
                    entitydict[j.get('entity2')]+=";"+j.get('target2_type')
            else:
                entitydict[j.get('entity2')]=j.get('target2_type')

In [67]:
len(entitydict)

6922

In [68]:
entitydict_tolist={}
for i,j in enumerate(triplelist):
    if j.get('entity1') in entity_set and j.get('entity2') in entity_set:
        if not entitydict_tolist.get(j.get('entity1')) and not entitydict_tolist.get(j.get('entity2')):
            entitydict_tolist[j.get('entity1')]=[j.get('target1_type')]
            entitydict_tolist[j.get('entity2')]=[j.get('target2_type')]
        else:
            if entitydict_tolist.get(j.get('entity1')):
                if j.get('target1_type') in entitydict_tolist.get(j.get('entity1')):
                    pass
                else:
                    entitydict_tolist[j.get('entity1')]+=[j.get('target1_type')]
            else:
                entitydict_tolist[j.get('entity1')]=[j.get('target1_type')]
            if entitydict_tolist.get(j.get('entity2')):
                if j.get('target2_type') in  entitydict_tolist.get(j.get('entity2')) :
                    pass
                else:
                    entitydict_tolist[j.get('entity2')]+=[j.get('target2_type')]
            else:
                entitydict_tolist[j.get('entity2')]=[j.get('target2_type')]

In [69]:
entitydict.get('Lipopolysaccharides')

'Carbohydrate;Lipid;Toxin'

In [70]:
entitydict_tolist.get('Lipopolysaccharides')

['Carbohydrate', 'Lipid', 'Toxin']

In [71]:
len(entitydict)

6922

In [72]:
list(entitydict.items())[-30:-1]

[('CSH2', 'Gene'),
 ('NOC3L', 'Gene'),
 ('WT1', 'Gene'),
 ('SIX2', 'Gene'),
 ('SIX1', 'Gene'),
 ('LTA', 'Gene'),
 ('IL12A', 'Gene'),
 ('CEBPB', 'Gene'),
 ('CTH', 'Gene'),
 ('CXCL2', 'Gene'),
 ('HMGB2', 'Gene'),
 ('CCR1', 'Gene'),
 ('KIF5A', 'Gene'),
 ('HPS1', 'Gene'),
 ('SLC25A4', 'Gene'),
 ('Ototoxicity', 'Disease'),
 ('MED28', 'Gene'),
 ('WSB1', 'Gene'),
 ('APELA', 'Gene'),
 ('Trisomy', 'Disease'),
 ('Cytokines', 'Peptide;Protein'),
 ('rs63750001', 'Mutation'),
 ('rs7141087', 'Mutation'),
 ('BEGAIN', 'Gene'),
 ('GOLPH3', 'Gene'),
 ('SUPT5H', 'Gene'),
 ('HOXA3', 'Gene'),
 ('G6PC1', 'Gene'),
 ('OSR1', 'Gene')]

In [73]:
for i,j in entitydict.items():
    if re.search(';',j):
        print({i:j})

{'Ceramides': 'Carbohydrate;Lipid'}
{'Ristocetin': 'Carbohydrate;Peptide'}
{'Lipopolysaccharides': 'Carbohydrate;Lipid;Toxin'}
{'Sphingomyelins': 'Carbohydrate;Lipid'}
{'Daptomycin': 'Lipid;Peptide'}
{'Vancomycin': 'Carbohydrate;Peptide'}
{'Bleomycin': 'Carbohydrate;Peptide'}
{'Glycosylphosphatidylinositols': 'Carbohydrate;Lipid'}
{'Abciximab': 'Peptide;Protein'}
{'Angiotensin II': 'Peptide;Protein'}
{'Angiotensins': 'Peptide;Protein'}
{'Psychosine': 'Carbohydrate;Lipid'}
{'Glycolipids': 'Carbohydrate;Lipid'}
{'Gangliosides': 'Carbohydrate;Lipid'}
{'G(M1) Ganglioside': 'Carbohydrate;Lipid'}
{'G(M3) Ganglioside': 'Carbohydrate;Lipid'}
{'Glycosphingolipids': 'Carbohydrate;Lipid'}
{'Phospholipid Ethers': 'Carbohydrate;Lipid'}
{'Glycopeptides': 'Carbohydrate;Peptide'}
{'Cilastatin, Imipenem Drug Combination': 'Lipid;Pharmaceutical Preparations'}
{'Phalloidine': 'Peptide;Toxin'}
{'Globosides': 'Carbohydrate;Lipid'}
{'Glucosylceramides': 'Carbohydrate;Lipid'}
{'Cerebrosides': 'Carbohydrate;L

In [74]:
entitynum={}
num=1
for i in entitydict.keys():
    entitynum[i]=num
    num+=1

In [75]:
entitynum.get('APOE')

243

In [76]:
for i in triplelistnew:
    i['verb']=i.get('triple')[1]

In [77]:
entity_countnum={}
relation_countnum={}
tripleWeight={}
tripleWeightunique=set()
for i in triplelistnew:
    if i.get('entity_one_transformed')!=i.get('entity_two_transformed'):
        if (i.get('entity_one_transformed'),i.get('triple')[1].lower(),i.get('entity_two_transformed'),i.get('PMID')) not in tripleWeightunique:
            tripleWeightunique.add((i.get('entity_one_transformed'),i.get('triple')[1].lower(),i.get('entity_two_transformed'),i.get('PMID')))
            entity_countnum[i.get('entity_one_transformed')]=entity_countnum.get(i.get('entity_one_transformed'),0)+1
            entity_countnum[i.get('entity_two_transformed')]=entity_countnum.get(i.get('entity_two_transformed'),0)+1
            relation_countnum[i.get('triple')[1]]=relation_countnum.get(i.get('triple')[1],0)+1
            tripleWeight[(i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed'))]=tripleWeight.get((i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed')),0)+1

In [78]:
len(entity_countnum)

6922

In [79]:
len(tripleWeight)

116487

In [80]:
len(triplelistnew)

146384

In [81]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [82]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
def cal_date(i):
    j=i
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    return date

In [83]:
for i in triplelistnew:
    if jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower()):
        i['IF']=jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower())
    elif jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower()):
        i['IF']=jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower())
    else:
        i['IF']=0.0
    if jt2if5_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower()):
        i['IF5']=jt2if5_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower())
    elif jt2if5_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower()):
        i['IF5']=jt2if5_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower())
    else:
        i['IF5']=0.0

In [84]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [85]:
relation_interpolation_list=[]
relation_interpolation_dict={}
for i in triplelistnew:
    if (i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed')) not in relation_interpolation_dict.keys():
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]={'from':entitynum.get(i.get('entity_one_transformed'))-1,
                                   'to':entitynum.get(i.get('entity_two_transformed'))-1,'relation':i.get('triple')[1],
                                   'PMID':[i.get('PMID')],'sentence':[i.get('sentence')],
                                   'DP':[pubtator_info_dict.get(i.get('PMID')).get('DP')],
                                   'TI':[pubtator_info_dict.get(i.get('PMID')).get('TI')],
                                    'TA':[pubtator_info_dict.get(i.get('PMID')).get('TA')],
                                    'IF':[i.get('IF')],   'IF5':[i.get('IF5')],   
                                    'date':[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))],
                                    'first_entity_transformed':i.get('entity_one_transformed'),'second_entity_transformed':i.get('entity_two_transformed'),                                                     
                                    'first_entity':[i.get('entity_one')],'second_entity':[i.get('entity_two')],
                                    'first_type':entitydict_tolist.get(i.get('entity_one_transformed')),'second_type':entitydict_tolist.get(i.get('entity_two_transformed')),
                                     'method':[', '.join(i.get('method'))]
                                                                                                                           }
    else:
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['PMID']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('PMID')+[i.get('PMID')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['sentence']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('sentence')+[i.get('sentence')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['first_entity']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('first_entity')+[i.get('entity_one')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['second_entity']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('second_entity')+[i.get('entity_two')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['DP']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('DP')+[pubtator_info_dict.get(i.get('PMID')).get('DP')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['TI']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('TI')+[pubtator_info_dict.get(i.get('PMID')).get('TI')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['TA']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('TA')+[pubtator_info_dict.get(i.get('PMID')).get('TA')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['date']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('date')+[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['IF']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('IF')+[i.get('IF')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['IF5']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('IF5')+[i.get('IF5')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['method']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('method')+[', '.join(i.get('method'))]

In [86]:
list(relation_interpolation_dict.items())[10]

(('Sarcopenia', 'associated', 'Constipation'),
 {'from': 3,
  'to': 13,
  'relation': 'associated',
  'PMID': ['34769606'],
  'sentence': ['(4) Conclusions: Sarcopenia and slow gait speed associated with constipation in community-dwelling older adults.'],
  'DP': ['2021 Oct 21'],
  'TI': ['Status of Constipation and Its Association with Sarcopenia in Older Adults: A Population-Based Cohort Study.'],
  'TA': ['Int J Environ Res Public Health'],
  'IF': [0.0],
  'IF5': [0.0],
  'date': [20211021],
  'first_entity_transformed': 'Sarcopenia',
  'second_entity_transformed': 'Constipation',
  'first_entity': ['Sarcopenia'],
  'second_entity': ['constipation'],
  'first_type': ['Disease'],
  'second_type': ['Disease'],
  'method': ['open domain']})

In [87]:
len(relation_interpolation_dict)

116495

In [88]:
relation_interpolation_list=list(relation_interpolation_dict.values())

In [89]:
len(relation_interpolation_list)

116495

In [90]:
entity2content={}
for i,j in enumerate(triplelistnew):
    if j.get('entity_one_transformed') in entity2content.keys():
        entity2content[j.get('entity_one_transformed')]['sentence']=entity2content.get(j.get('entity_one_transformed')).get('sentence')+[j.get('sentence')]
        entity2content[j.get('entity_one_transformed')]['PMID']=entity2content.get(j.get('entity_one_transformed')).get('PMID')+[j.get('PMID')]
        entity2content[j.get('entity_one_transformed')]['entity']=entity2content.get(j.get('entity_one_transformed')).get('entity')+[j.get('entity_one')]
        entity2content[j.get('entity_one_transformed')]['DP']=entity2content.get(j.get('entity_one_transformed')).get('DP')+[pubtator_info_dict.get(j.get('PMID')).get('DP')]
        entity2content[j.get('entity_one_transformed')]['TI']=entity2content.get(j.get('entity_one_transformed')).get('TI')+[pubtator_info_dict.get(j.get('PMID')).get('TI')]
        entity2content[j.get('entity_one_transformed')]['TA']=entity2content.get(j.get('entity_one_transformed')).get('TA')+[pubtator_info_dict.get(j.get('PMID')).get('TA')]
        entity2content[j.get('entity_one_transformed')]['date']=entity2content.get(j.get('entity_one_transformed')).get('date')+[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))]
        entity2content[j.get('entity_one_transformed')]['IF']=entity2content.get(j.get('entity_one_transformed')).get('IF')+[j.get('IF')]
        entity2content[j.get('entity_one_transformed')]['IF5']=entity2content.get(j.get('entity_one_transformed')).get('IF5')+[j.get('IF5')]
        entity2content[j.get('entity_one_transformed')]['method']=entity2content.get(j.get('entity_one_transformed')).get('method')+[', '.join(j.get('method'))]
    else:
        entity2content[j.get('entity_one_transformed')]={'entity_transformed':j.get('entity_one_transformed'),'entity':[j.get('entity_one')],'sentence':[j.get('sentence')],'PMID':[j.get('PMID')],
                                                         'DP':[pubtator_info_dict.get(j.get('PMID')).get('DP')],'TI':[pubtator_info_dict.get(j.get('PMID')).get('TI')],
                                                        'TA':[pubtator_info_dict.get(j.get('PMID')).get('TA')],'date':[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))],
                                                        'IF':[j.get('IF')],'IF5':[j.get('IF5')],'method':[', '.join(j.get('method'))]}
    if j.get('entity_two_transformed') in entity2content.keys():
        entity2content[j.get('entity_two_transformed')]['sentence']=entity2content.get(j.get('entity_two_transformed')).get('sentence')+[j.get('sentence')]
        entity2content[j.get('entity_two_transformed')]['PMID']=entity2content.get(j.get('entity_two_transformed')).get('PMID')+[j.get('PMID')]
        entity2content[j.get('entity_two_transformed')]['entity']=entity2content.get(j.get('entity_two_transformed')).get('entity')+[j.get('entity_two')]
        entity2content[j.get('entity_two_transformed')]['DP']=entity2content.get(j.get('entity_two_transformed')).get('DP')+[pubtator_info_dict.get(j.get('PMID')).get('DP')]
        entity2content[j.get('entity_two_transformed')]['TI']=entity2content.get(j.get('entity_two_transformed')).get('TI')+[pubtator_info_dict.get(j.get('PMID')).get('TI')]
        entity2content[j.get('entity_two_transformed')]['TA']=entity2content.get(j.get('entity_two_transformed')).get('TA')+[pubtator_info_dict.get(j.get('PMID')).get('TA')]
        entity2content[j.get('entity_two_transformed')]['date']=entity2content.get(j.get('entity_two_transformed')).get('date')+[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))]
        entity2content[j.get('entity_two_transformed')]['IF']=entity2content.get(j.get('entity_two_transformed')).get('IF')+[j.get('IF')]
        entity2content[j.get('entity_two_transformed')]['IF5']=entity2content.get(j.get('entity_two_transformed')).get('IF5')+[j.get('IF5')]
        entity2content[j.get('entity_two_transformed')]['method']=entity2content.get(j.get('entity_two_transformed')).get('method')+[', '.join(j.get('method'))]
    else:
        entity2content[j.get('entity_two_transformed')]={'entity_transformed':j.get('entity_two_transformed'),'entity':[j.get('entity_two')],'sentence':[j.get('sentence')],'PMID':[j.get('PMID')],
                                                         'DP':[pubtator_info_dict.get(j.get('PMID')).get('DP')],'TI':[pubtator_info_dict.get(j.get('PMID')).get('TI')],
                                                        'TA':[pubtator_info_dict.get(j.get('PMID')).get('TA')],'date':[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))],
                                                        'IF':[j.get('IF')],'IF5':[j.get('IF5')],'method':[', '.join(j.get('method'))]}

In [91]:
len(entity2content)

6922

In [92]:
entity2list=list(entity2content.values())

In [93]:
len(entity2list)

6922

In [94]:
for i in entity2list:
    i['_id']=i.get('entity_transformed')

In [95]:
entitydict.get(entity2list[0].get('entity_transformed'))

'Disease'

In [96]:
for i in entity2list:
    i['type']=entitydict.get(i.get('entity_transformed')).split(';')

In [97]:
len(entity2list)

6922

In [98]:
json.dump(entity2list,open('results/step4/entityinfo.json','w'))

In [99]:
for i in entity2list:
    if i.get('entity_transformed')=='HMGB1':
        print(i)

{'entity_transformed': 'HMGB1', 'entity': ['HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1', 'HMGB1'], 'sentence': ['In an in vitro study we used the 18-beta-stereoisomer of GTA to enhance Sirt6 expression levels, inhibiting through this mechanism the translocation of HMGB1 protein from nucleus and reversing its extracellular accumulation stimulated by lipopolysaccharides.', 'In an in vitro study we used the 18-beta-stereoisomer of GTA to enhance Sirt6 expression levels, inhibiting through this mechanism the translocation of HMGB1 protein from nucleus and reversing its extracellular accumulation stimulated by lipopolysaccharides.', 'In an in vitro study we used the 18-beta-stereoisomer of GTA to enhance Sirt6 expression levels, inhibiting through this mechanism

In [100]:
entity2list[-1]

{'entity_transformed': 'Saxitoxin',
 'entity': ['Saxitoxin'],
 'sentence': ['Saxitoxin (STX) causes high toxicity by blocking voltage-gated sodium channels, and it poses a major threat to marine ecosystems and human health worldwide.'],
 'PMID': ['37888479'],
 'DP': ['2023 Oct 19'],
 'TI': ['Physiological Effects of Oxidative Stress Caused by Saxitoxin in the Nematode Caenorhabditis elegans.'],
 'TA': ['Mar Drugs'],
 'date': [20231019],
 'IF': [0.0],
 'IF5': [0.0],
 'method': ['shortest path'],
 '_id': 'Saxitoxin',
 'type': ['Toxin']}

In [101]:
for i in entity2list:
    if i.get('entity_transformed')=='Lipopolysaccharides':
        print(i.get('type'))

['Carbohydrate', 'Lipid', 'Toxin']


In [102]:
list(relation_interpolation_dict.items())[:3]

[(('Pulmonary Disease, Chronic Obstructive', 'defined', 'Inflammation'),
  {'from': 0,
   'to': 1,
   'relation': 'defined',
   'PMID': ['30781849'],
   'sentence': ['(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.'],
   'DP': ['2019 Feb 13'],
   'TI': ['Chronic Obstructive Pulmonary Disease as a Main Factor of Premature Aging.'],
   'TA': ['Int J Environ Res Public Health'],
   'IF': [0.0],
   'IF5': [0.0],
   'date': [20190213],
   'first_entity_transformed': 'Pulmonary Disease, Chronic Obstructive',
   'second_entity_transformed': 'Inflammation',
   'first_entity': ['COPD'],
   'second_entity': ['inflammatory disorder'],
   'first_type': ['Disease'],
   'second_type': ['Disease'],
   'method': ['deep learning, shortest path']}),
 (('Anorexia', 'associate', 'Sarcopenia'),
  {'from': 2,
   'to': 3,
   'relation': 'associate',
   'PMID': ['30641897'],
   'sentence': ["(1) Backgro

In [103]:
relation_interpolation_list[10]

{'from': 3,
 'to': 13,
 'relation': 'associated',
 'PMID': ['34769606'],
 'sentence': ['(4) Conclusions: Sarcopenia and slow gait speed associated with constipation in community-dwelling older adults.'],
 'DP': ['2021 Oct 21'],
 'TI': ['Status of Constipation and Its Association with Sarcopenia in Older Adults: A Population-Based Cohort Study.'],
 'TA': ['Int J Environ Res Public Health'],
 'IF': [0.0],
 'IF5': [0.0],
 'date': [20211021],
 'first_entity_transformed': 'Sarcopenia',
 'second_entity_transformed': 'Constipation',
 'first_entity': ['Sarcopenia'],
 'second_entity': ['constipation'],
 'first_type': ['Disease'],
 'second_type': ['Disease'],
 'method': ['open domain']}

In [104]:
len(relation_interpolation_list)

116495

In [105]:
json.dump(relation_interpolation_list,open('results/step4/relationinfo.json','w'))

In [106]:
relation_interpolation_list[100]

{'from': 5,
 'to': 96,
 'relation': 'associated',
 'PMID': ['34772307'],
 'sentence': ['Drowsiness/lethargy (OR = 1.48, 95% CI: 1.22-1.82), agitation (OR = 1.66, 95% CI: 1.11-2.50), coma (OR = 23.95, 95% CI: 17.05-33.64), bradycardia (OR = 2.29, 95% CI: 1.22-4.32), rhabdomyolysis (OR = 8.84, 95% CI: 3.71-21.03), hypothermia (OR = 4.1, 95% CI: 1.77-9.51), and hyperthermia 2.10 (OR = 2.10, 95% CI: 1.04-4.22) were likely associated with major outcomes or death.'],
 'DP': ['2021 Dec'],
 'TI': ['Prognostic factors of acetaminophen exposure in the United States: An analysis of 39,000 patients.'],
 'TA': ['Hum Exp Toxicol'],
 'IF': ['2.8'],
 'IF5': ['2.9'],
 'date': [20211201],
 'first_entity_transformed': 'Death',
 'second_entity_transformed': 'Rhabdomyolysis',
 'first_entity': ['death'],
 'second_entity': ['rhabdomyolysis'],
 'first_type': ['Disease'],
 'second_type': ['Disease'],
 'method': ['shortest path']}

In [107]:
len(relation_interpolation_list)

116495

In [108]:
len(triplelistnew)

146384

In [109]:
csvf_entity = open("results/step4/entity.csv", "w", newline='', encoding='utf-8')
w_entity = csv.writer(csvf_entity)

In [110]:
w_entity.writerow(("entity:ID", "name","type","frequency",":LABEL"))

38

In [111]:
for i,j in entitydict.items():
    w_entity.writerow((entitynum.get(i), i,j,int(entity_countnum.get(i)),j))

In [112]:
csvf_entity.close()

In [113]:
entitycsv=pd.read_csv('results/step4/entity.csv')

In [114]:
entitycsv[:3]

,entity:ID,name,type,frequency,:LABEL
0,1,"Pulmonary Disease, Chronic Obstructive",Disease,1034,Disease
1,2,Inflammation,Disease,4175,Disease
2,3,Anorexia,Disease,147,Disease


In [115]:
len(entitycsv)

6922

In [116]:
csvf_entity = open("results/step4/roles.csv", "w", newline='', encoding='utf-8')
w_entity = csv.writer(csvf_entity)

In [117]:
w_entity.writerow((':START_ID', ':END_ID', "relation", "weight", "method", ":TYPE" ))

48

In [118]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [119]:
entitynotsame={}
for i in triplelistnew:
    if i.get('entity_one_transformed')!=i.get('entity_two_transformed'):
        if not entitynotsame.get((i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed'))) :
            w_entity.writerow((  entitynum.get(i.get('entity_one_transformed')),entitynum.get(i.get('entity_two_transformed')), i.get('triple')[1].lower(),int(tripleWeight.get((i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed')))),'; '.join(i.get('method')), i.get('triple')[1]   ))
        else:
            continue
        entitynotsame[(i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed'))]=True

In [120]:
entitynum.get('APOE')

243

In [121]:
csvf_entity.close()

In [122]:
for i in triplelistnew:
    if i.get('sentence')=='Incidence of epilepsy was not statistically elevated among males, those with dementia, or individuals reporting a history of head injury or treatment for depression.':
        print(i)

{'triple': ['dementia', 'report', 'head injury'], 'PMID': '16870396', 'sentence': 'Incidence of epilepsy was not statistically elevated among males, those with dementia, or individuals reporting a history of head injury or treatment for depression.', 'AB': 'OBJECTIVE: To determine age-specific incidence and cumulative incidence of epilepsy in a well-defined cohort of elderly people, and to examine how rates of epilepsy are modified by sex, race, stroke, dementia, head injury, and depression. METHODS: The authors examined data from a reconstructed cohort based on 1919 community-dwelling volunteers, followed as part of a large ongoing prospective aging study. RESULTS: Age-specific incidence was 10.6 (per 100,000 person-years) between ages 45 and 59, 25.8 between ages 60 and 74, and 101.1 between ages 75 and 89. Cumulative incidence was 0.15% from age 45 to age 60, 0.38% to age 70, 1.01% to age 80, and 1.47% to age 90. In addition, the difference in cumulative incidence among African-Amer

In [123]:
roles=pd.read_csv("results/step4/roles.csv")

In [124]:
len(roles)

116487

In [125]:
roles.head()

,:START_ID,:END_ID,relation,weight,method,:TYPE
0,1,2,defined,1,deep learning; shortest path,defined
1,3,4,associate,1,deep learning,associate
2,4,3,associate,1,deep learning,associate
3,5,6,recognized,1,deep learning,recognized
4,5,6,increase,1,shortest path,increase


In [126]:
relation_list=roles.relation.to_list()

In [127]:
type(relation_list)

list

In [128]:
relation_list[:3]

['defined', 'associate', 'associate']

In [129]:
len(relation_list)

116487

In [130]:
len(set(relation_list))

3027

In [131]:
relation_list_set=pd.read_csv('../classofrelations/relation_list_set.csv',header=None,names=['word'])

In [132]:
relation_list_set_dict={}
for i in relation_list_set['word'].to_list():
    relation_list_set_dict[i.lower()]=True

In [133]:
positive_relations=pd.read_csv('../classofrelations/positive-relations.csv',header=None,names=['word'])

In [134]:
positive_relations_dict={}
for i in positive_relations['word'].to_list():
    positive_relations_dict[i.lower()]=True

In [135]:
negative_relations=pd.read_csv('../classofrelations/negative-relations.csv',header=None,names=['word'])

In [136]:
negative_relations_dict={}
for i in negative_relations['word'].to_list():
    negative_relations_dict[i.lower()]=True

In [137]:
negative_relations['word'].to_list()[-5:-1]

['rescued', 'restrained', 'reduce complications in', 'reduce relative in']

In [138]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [139]:
dropout=[]
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                if positive_relations_dict.get(i.get('triple')[1].lower()):
                    i['partofspeech']='positive'
                elif negative_relations_dict.get(i.get('triple')[1].lower()):
                    i['partofspeech']='negative'
                else:
                    i['partofspeech']='none'
                    dropout.append(i)

In [140]:
len(dropout)

9794

In [141]:
relation_list=[]
disease_list_raw=[]
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                relation_list.append(i.get('triple')[1].lower())
                disease_list_raw.append(i)

In [142]:
len(relation_list)

19691

In [143]:
len(disease_list_raw)

19691

In [144]:
new_relation=set()
for i in disease_list_raw:
    if not relation_list_set_dict.get(i.get('triple')[1].lower()):
        new_relation.add(i.get('triple')[1].lower())

In [145]:
len(new_relation)

7

In [146]:
for i in disease_list_raw:
    if i.get('triple')[1].lower()=='with those be':
        print(i.get('triple'))
        print(i.get('sentence'))
        # break

['CD4', 'with those be', 'neuroretinal disease']
RESULTS: Ocular manifestations from any cause develop from 35 years of age in patients with HIV/AIDS, with the highest risk for age-related macular degeneration over the fourth decade of life and for the development of neuroretinal disorder on the fifth decade of life; some studies report a slight tendency to diagnose macular degeneration in women and those who acquired AIDS through sexual contact; data contrasted with increased risk for diagnosing neuroretinal disorder in homosexual men who also use intravenous drugs, possibly due to oversampling in studies; non-Hispanic whites and African Americans were the races most commonly affected by neuroretinal disease; the means between the 11.3-14.5 years elapsed since the HIV diagnosis were more frequently associated with cognitive impairment and both in those with high or low CD4 counts, and in patients with high or low viral loads, neuroretinal disease without Statistically significant diff

In [147]:
new_relation

{'assist', 'boost', 'compares', 'oversee', 'posed', 'saturated', 'tied'}

In [148]:
len(relation_list_set['word'].to_list())

1721

In [149]:
relation_list_set_new=sorted(relation_list_set['word'].to_list())
for i in new_relation:
    relation_list_set_new.append(i)
pd.DataFrame(relation_list_set_new,columns=['word']).to_csv('../classofrelations/relation_list_set.csv')

In [150]:
pd.DataFrame(relation_list_set_new,columns=['word']).to_csv('results/step4/relation_list_set.csv')

# DEAL WITH new_relation!!!!!!!!!!!!
## Classify the new_relation list into posit, negat, and middle lists.

In [157]:
posit=[]
negat=['assist']
# middle=['blocked','drifting','engage','expanded','hyperactivated','incorporated','mistaken','processed',
#        'reformed','relapsed','span','supplemented']

In [158]:
for i in disease_list_raw:
    if i.get('triple')[1]=='tied':
        print(i)
        break

{'triple': ['AMPK', 'tied', 'death'], 'PMID': '38022638', 'sentence': 'The silent mating type information regulation 2 homolog 1 (Saccharomyces cerevisiae) (SIRT1), AMP activated protein kinase (AMPK), and Wnt1 inducible signaling pathway protein 1 (WISP1) are novel targets that can oversee programmed cell death pathways tied to beta-nicotinamide adenine dinucleotide (NAD+), nicotinamide, apolipoprotein E (APOE), severe acute respiratory syndrome (SARS-CoV-2) exposure with coronavirus disease 2019 (COVID-19), and trophic factors, such as erythropoietin (EPO).', 'AB': 'Life expectancy is increasing throughout the world and coincides with a rise in non-communicable diseases (NCDs), especially for metabolic disease that includes diabetes mellitus (DM) and neurodegenerative disorders. The debilitating effects of metabolic disorders influence the entire body and significantly affect the nervous system impacting greater than one billion people with disability in the peripheral nervous system

In [159]:
positive_relations_new=positive_relations['word'].to_list()
negative_relations_new=negative_relations['word'].to_list()

In [160]:
positive_relations_new=positive_relations['word'].to_list()
for i in posit:
    positive_relations_new.append(i)
pd.DataFrame(positive_relations_new,columns=['word']).to_csv('../classofrelations/positive-relations.csv')

In [161]:
negative_relations_new=negative_relations['word'].to_list()
for i in negat:
    negative_relations_new.append(i)
pd.DataFrame(negative_relations_new,columns=['word']).to_csv('../classofrelations/negative-relations.csv')

In [162]:
pd.DataFrame(positive_relations_new,columns=['word']).to_csv('results/step4/positive-relations.csv')
pd.DataFrame(negative_relations_new,columns=['word']).to_csv('results/step4/negative-relations.csv')

In [163]:
print(len(positive_relations_new))
print(len(negative_relations_new))

328
272


In [172]:
negative_relations_new[-5:]

['restrained',
 'reduce complications in',
 'reduce relative in',
 'show activity without',
 'assist']

In [165]:
positive_relations_new_dict={}
for i in positive_relations_new:
    positive_relations_new_dict[i.lower()]=True

In [166]:
negative_relations_new_dict={}
for i in negative_relations_new:
    negative_relations_new_dict[i.lower()]=True

In [167]:
dropout=[]
disease_list=[]
for i in disease_list_raw:
    if positive_relations_new_dict.get(i.get('triple')[1].lower()):
        i['partofspeech']='positive'
        disease_list.append(i)
    elif negative_relations_new_dict.get(i.get('triple')[1].lower()):
        i['partofspeech']='negative'
        disease_list.append(i)
    else:
        i['partofspeech']='none'
        dropout.append(i)

In [168]:
len(disease_list)

9898

In [173]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i in disease_list:
    j=pubtator_info_dict.get(i.get('PMID')).get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['TA']=pubtator_info_dict.get(i.get('PMID')).get('TA')
    i['JT']=pubtator_info_dict.get(i.get('PMID')).get('JT')
    i['year']=int(str(date)[:4])

In [174]:
disease_list[-1]

{'triple': ['Saxitoxin', 'cause', 'toxicity'],
 'PMID': '37888479',
 'sentence': 'Saxitoxin (STX) causes high toxicity by blocking voltage-gated sodium channels, and it poses a major threat to marine ecosystems and human health worldwide.',
 'AB': 'Saxitoxin (STX) causes high toxicity by blocking voltage-gated sodium channels, and it poses a major threat to marine ecosystems and human health worldwide. Our work evaluated the neurotoxicity and chronic toxicology of STX to Caenorhabditis elegans by an analysis of lifespan, brood size, growth ability, reactive oxygen species (ROS) and adenosine triphosphate (ATP) levels, and the overexpression of green fluorescent protein (GFP). After exposure to a series of concentrations of STX for 24 h, worms showed paralysis symptoms and fully recovered within 6 h; less than 5% of worms died at the highest concentration of 1000 ng/mL for first larval stage (L1) worms and 10,000 ng/mL for fourth larval stage (L4) worms. Declines in lifespan, productivi

In [175]:
for i in disease_list:
    if i.get('triple')[1]=='impede':
        print(i)
        break

{'triple': ['NOTCH1', 'impede', 'carcinogenesis'], 'PMID': '36658434', 'sentence': 'NOTCH1 mutant clones occupy the majority of normal human esophagus by middle age but are comparatively rare in esophageal cancers, suggesting NOTCH1 mutations drive clonal expansion but impede carcinogenesis.', 'AB': 'NOTCH1 mutant clones occupy the majority of normal human esophagus by middle age but are comparatively rare in esophageal cancers, suggesting NOTCH1 mutations drive clonal expansion but impede carcinogenesis. Here we test this hypothesis. Sequencing NOTCH1 mutant clones in aging human esophagus reveals frequent biallelic mutations that block NOTCH1 signaling. In mouse esophagus, heterozygous Notch1 mutation confers a competitive advantage over wild-type cells, an effect enhanced by loss of the second allele. Widespread Notch1 loss alters transcription but has minimal effects on the epithelial structure and cell dynamics. In a carcinogenesis model, Notch1 mutations were less prevalent in tu

In [176]:
len(disease_list)

9898

In [177]:
entity_pairs=['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [178]:
names=locals()
for i in entity_pairs:
    names[i+'_typecount']=set()
for j in disease_list:
    for i in entitydict_tolist.get(j.get('entity_one_transformed')):
        i=i.replace(' ','_')
        names[i+'_typecount'].add(j.get('entity_one_transformed'))

In [179]:
entitydict_tolist.get('Lipopolysaccharides')

['Carbohydrate', 'Lipid', 'Toxin']

In [180]:
for i in entity_pairs:
    print(i+' '+str(len(list(names[i+'_typecount']))))

Gene 1492
RNA 43
Protein 18
Carbohydrate 82
Lipid 84
Peptide 27
Pharmaceutical_Preparations 3
Toxin 4
Mutation 293
Disease 0


In [181]:
disease_list_copy=copy.deepcopy(disease_list)

In [182]:
disease_list_copy[0]

{'triple': ['alanine aminotransferase', 'recognized', 'mortality'],
 'PMID': '34063029',
 'sentence': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty.',
 'AB': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty. In the elderly, both frailty and ischemic stroke are not only common, but are also associated with mortality. The aim of this research was to investigate whether a significantly reduced ALT level increases the all-cause mortality rate in the elderly with ischemic stroke. (2) Methods: Between February 2014 and April 2019, a retrospective study of 901 patients with ischemic stroke admitted to a university-affiliated hospital was conducted. Cox proportional hazard regression was used to determine whether a significantly reduced AL

In [183]:
for i in disease_list_copy:
    if len(entitydict_tolist.get(i.get('entity_one_transformed')))>1:
        i['entity_one_type']='; '.join(entitydict_tolist.get(i.get('entity_one_transformed')))

In [184]:
list(entitydict_tolist.items())[0]

('Pulmonary Disease, Chronic Obstructive', ['Disease'])

In [185]:
disease_list_copy[0]

{'triple': ['alanine aminotransferase', 'recognized', 'mortality'],
 'PMID': '34063029',
 'sentence': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty.',
 'AB': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty. In the elderly, both frailty and ischemic stroke are not only common, but are also associated with mortality. The aim of this research was to investigate whether a significantly reduced ALT level increases the all-cause mortality rate in the elderly with ischemic stroke. (2) Methods: Between February 2014 and April 2019, a retrospective study of 901 patients with ischemic stroke admitted to a university-affiliated hospital was conducted. Cox proportional hazard regression was used to determine whether a significantly reduced AL

In [186]:
len(disease_list_copy)

9898

In [187]:
positive_relations=[]
negative_relations=[]
for i in disease_list_copy:
    if i.get('partofspeech')=='positive':
        positive_relations.append(i)
    else:
        negative_relations.append(i)

In [188]:
len(positive_relations)

8416

In [189]:
json.dump(positive_relations,open('results/step4/aging_relations.json','w'))
json.dump(negative_relations,open('results/step4/longevity_relations.json','w'))

In [190]:
json.dump(disease_list_copy,open('results/step4/disease_relation.json','w'))

In [191]:
positive_relations=json.load(open('results/step4/aging_relations.json','r'))
negative_relations=json.load(open('results/step4/longevity_relations.json','r'))
disease_list_copy=json.load(open('results/step4/disease_relation.json','r'))

# Statistics

In [192]:
for j in entity_pairs:
    j=j.replace('_',' ')
    names[j+'_aging']=set()
for i in disease_list:
    if i.get('partofspeech')=='positive':
        for j in entitydict_tolist.get(i.get('entity_one_transformed')):
            j=j.replace('_',' ')
            names[j+'_aging'].add(i.get('entity_one_transformed'))
for j in entity_pairs:    
    j=j.replace('_',' ')
    print(j+'\t'+str(len(names[j+'_aging'])))

Gene	1368
RNA	39
Protein	17
Carbohydrate	70
Lipid	75
Peptide	22
Pharmaceutical Preparations	2
Toxin	4
Mutation	289
Disease	0


In [193]:
for j in entity_pairs:    
    j=j.replace('_',' ')
    print(str(len(names[j+'_aging']))+',', end='')

1368,39,17,70,75,22,2,4,289,0,

In [194]:
for j in entity_pairs:
    j=j.replace('_',' ')
    names[j+'_longevity']=set()
for i in disease_list:
    if i.get('partofspeech')=='negative':
        for j in entitydict_tolist.get(i.get('entity_one_transformed')):
            j=j.replace('_',' ')
            names[j+'_longevity'].add(i.get('entity_one_transformed'))
for j in entity_pairs:    
    j=j.replace('_',' ')
    print(j+'\t'+str(len(names[j+'_longevity'])))

Gene	400
RNA	12
Protein	9
Carbohydrate	53
Lipid	40
Peptide	13
Pharmaceutical Preparations	3
Toxin	1
Mutation	8
Disease	0


In [195]:
Genedict=json.load(open('../dict/Genedict.json'))

In [196]:
list(Genedict.items())[:10]

[('A1BG', ['A1BG']),
 ('A1CF', ['A1CF']),
 ('A2M', ['A2M']),
 ('A2ML1', ['A2ML1']),
 ('A3GALT2', ['A3GALT2']),
 ('A4GALT', ['A4GALT']),
 ('A4GNT', ['A4GNT']),
 ('AAAS', ['AAAS']),
 ('AACS', ['AACS']),
 ('AADAC', ['AADAC'])]

In [197]:
Genedict_upper={}
for i,j in Genedict.items():
    Genedict_upper[i.upper()]=j

In [198]:
num_of_aging=set()
num_of_longevity=set()
for i in disease_list:
    if i.get('partofspeech')=='positive':
        num_of_aging.add(i.get('entity_one_transformed'))
    elif i.get('partofspeech')=='negative':
        num_of_longevity.add(i.get('entity_one_transformed'))
print(len(num_of_aging))
print(len(num_of_longevity))

1871
531


In [199]:
disease_entity_list=[]
disease_entity_triple=set()
disease_entity_dict={}
for i in disease_list:
    if i.get('partofspeech')=='positive':
        if i.get('entity_one_type')=='Gene':
            disease_entity_triple.add((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')))
            disease_entity_list.append(i)
            disease_entity_dict[i.get('entity_one_transformed')]=disease_entity_dict.get(i.get('entity_one_transformed'),0)+1

In [200]:
len(disease_entity_dict)

1368

In [201]:
disease_entity_dict_sorted = sorted(disease_entity_dict.items(), key=lambda x: x[1], reverse=True)

In [202]:
len(disease_entity_dict_sorted)

1368

In [203]:
for i in disease_entity_dict_sorted[:20]:
    print(i)

('INS', 474)
('CRP', 241)
('APOE', 181)
('KL', 180)
('MAPT', 165)
('SIRT1', 103)
('IL6', 94)
('ADIPOQ', 80)
('CD4', 76)
('LEP', 70)
('LMNA', 69)
('ALB', 65)
('WRN', 62)
('TP53', 57)
('SIRT6', 46)
('ACE', 43)
('MTOR', 42)
('CST3', 41)
('SHBG', 39)
('BDNF', 36)


In [204]:
relation_interpolation_list[0]

{'from': 0,
 'to': 1,
 'relation': 'defined',
 'PMID': ['30781849'],
 'sentence': ['(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.'],
 'DP': ['2019 Feb 13'],
 'TI': ['Chronic Obstructive Pulmonary Disease as a Main Factor of Premature Aging.'],
 'TA': ['Int J Environ Res Public Health'],
 'IF': [0.0],
 'IF5': [0.0],
 'date': [20190213],
 'first_entity_transformed': 'Pulmonary Disease, Chronic Obstructive',
 'second_entity_transformed': 'Inflammation',
 'first_entity': ['COPD'],
 'second_entity': ['inflammatory disorder'],
 'first_type': ['Disease'],
 'second_type': ['Disease'],
 'method': ['deep learning, shortest path']}

In [205]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [206]:
direction=set()
direction_list=[]
direction_type=[]
for i in triplelistnew:
    for j in entitydict_tolist.get(i.get('entity_one_transformed')):
        for k in entitydict_tolist.get(i.get('entity_two_transformed')):
            if (i.get('entity_one_transformed'),(i.get('entity_two_transformed')),j,k) not in direction:
                direction.add((i.get('entity_one_transformed'),(i.get('entity_two_transformed')),j,k))
                direction_list.append((i.get('entity_one_transformed'),(i.get('entity_two_transformed'))))
                direction_type.append((j.replace(' ','_'),k.replace(' ','_')))

In [207]:
len(direction_list)

60203

In [208]:
type_dict={}
type_dict_ensure={}
for i in range(len(direction_list)):
    if not type_dict_ensure.get((direction_list[i][0],direction_list[i][1],direction_type[i][0],direction_type[i][1])):
        type_dict[(direction_list[i][0],direction_list[i][1])]=tuple((direction_type[i][0],direction_type[i][1]))
        type_dict_ensure[(direction_list[i][0],direction_list[i][1],direction_type[i][0],direction_type[i][1])]=tuple((direction_type[i][0],direction_type[i][1]))

In [209]:
len(type_dict)

59693

In [210]:
entity_pairs

['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [211]:
entity_pairs=['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [212]:
entity_pairs_dict={}
entity_pairs_dict_num={}
for i in entity_pairs:
    temp={}
    temp2={}
    for j in entity_pairs:
        temp.update({j:0})
        temp2.update({j:set()})
    entity_pairs_dict_num[i]=temp
    entity_pairs_dict[i]=temp2

In [213]:
entity_pairs_dict

{'Gene': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'RNA': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Protein': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Carbohydrate': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Lipid': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipi

In [214]:
for i in range(len(direction_list)):
        if direction_list[i][0] not in entity_pairs_dict.get(direction_type[i][1]).get(direction_type[i][0]):
            entity_pairs_dict[direction_type[i][1]][direction_type[i][0]].add(direction_list[i][0])
            entity_pairs_dict_num[direction_type[i][1]][direction_type[i][0]]+=1

In [215]:
entity_pairs_dict_num

{'Gene': {'Gene': 1253,
  'RNA': 39,
  'Protein': 10,
  'Carbohydrate': 57,
  'Lipid': 46,
  'Peptide': 24,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 2,
  'Mutation': 234,
  'Disease': 799},
 'RNA': {'Gene': 41,
  'RNA': 6,
  'Protein': 0,
  'Carbohydrate': 1,
  'Lipid': 0,
  'Peptide': 0,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 0,
  'Mutation': 1,
  'Disease': 39},
 'Protein': {'Gene': 38,
  'RNA': 0,
  'Protein': 4,
  'Carbohydrate': 5,
  'Lipid': 4,
  'Peptide': 0,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 0,
  'Mutation': 0,
  'Disease': 82},
 'Carbohydrate': {'Gene': 166,
  'RNA': 3,
  'Protein': 3,
  'Carbohydrate': 69,
  'Lipid': 38,
  'Peptide': 10,
  'Pharmaceutical_Preparations': 2,
  'Toxin': 1,
  'Mutation': 7,
  'Disease': 326},
 'Lipid': {'Gene': 184,
  'RNA': 5,
  'Protein': 2,
  'Carbohydrate': 28,
  'Lipid': 70,
  'Peptide': 6,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 1,
  'Mutation': 16,
  'Disease': 336},
 'Peptide': {'Gene': 54,
  'RNA': 0,
  

In [216]:
countnum=0
for i in entity_pairs_dict_num.values():
    for j in i.values():
        countnum+=j
print(countnum)

9793


In [217]:
relation_display=[]
for i,j in entity_pairs_dict_num.items():
    for k,l in j.items():
        relation_display.append([k,i+' ',l])

In [218]:
j

{'Gene': 2060,
 'RNA': 65,
 'Protein': 24,
 'Carbohydrate': 109,
 'Lipid': 112,
 'Peptide': 39,
 'Pharmaceutical_Preparations': 8,
 'Toxin': 6,
 'Mutation': 398,
 'Disease': 2334}

In [219]:
relation_display

[['Gene', 'Gene ', 1253],
 ['RNA', 'Gene ', 39],
 ['Protein', 'Gene ', 10],
 ['Carbohydrate', 'Gene ', 57],
 ['Lipid', 'Gene ', 46],
 ['Peptide', 'Gene ', 24],
 ['Pharmaceutical_Preparations', 'Gene ', 0],
 ['Toxin', 'Gene ', 2],
 ['Mutation', 'Gene ', 234],
 ['Disease', 'Gene ', 799],
 ['Gene', 'RNA ', 41],
 ['RNA', 'RNA ', 6],
 ['Protein', 'RNA ', 0],
 ['Carbohydrate', 'RNA ', 1],
 ['Lipid', 'RNA ', 0],
 ['Peptide', 'RNA ', 0],
 ['Pharmaceutical_Preparations', 'RNA ', 0],
 ['Toxin', 'RNA ', 0],
 ['Mutation', 'RNA ', 1],
 ['Disease', 'RNA ', 39],
 ['Gene', 'Protein ', 38],
 ['RNA', 'Protein ', 0],
 ['Protein', 'Protein ', 4],
 ['Carbohydrate', 'Protein ', 5],
 ['Lipid', 'Protein ', 4],
 ['Peptide', 'Protein ', 0],
 ['Pharmaceutical_Preparations', 'Protein ', 0],
 ['Toxin', 'Protein ', 0],
 ['Mutation', 'Protein ', 0],
 ['Disease', 'Protein ', 82],
 ['Gene', 'Carbohydrate ', 166],
 ['RNA', 'Carbohydrate ', 3],
 ['Protein', 'Carbohydrate ', 3],
 ['Carbohydrate', 'Carbohydrate ', 69],
 [

In [220]:
entity_pairs

['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [221]:
for i in entity_pairs:
    xs=[]
    for j in relation_display:
        if j[0]==i.replace(' ','_'):
            xs.append(str(j[-1]))
    print('\t'.join(xs))

1253	41	38	166	184	54	1	24	142	2060
39	6	0	3	5	0	0	2	1	65
10	0	4	3	2	1	0	0	0	24
57	1	5	69	28	5	0	0	2	109
46	0	4	38	70	12	0	2	8	112
24	0	0	10	6	9	0	1	1	39
0	0	0	2	0	1	0	0	0	8
2	0	0	1	1	1	0	2	0	6
234	1	0	7	16	1	0	0	94	398
799	39	82	326	336	110	14	31	161	2334


In [222]:
all_es=json.load(open('results/step2/all_es_lack.json'))

In [223]:
entity2aging={}
entity2longevity={}
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                if positive_relations_new_dict.get(i.get('triple')[1].lower()):
                    entity2aging[i.get('entity_one_transformed')]=True
                elif negative_relations_new_dict.get(i.get('triple')[1].lower()):
                    entity2longevity[i.get('entity_one_transformed')]=True

In [224]:
len(entity2aging)

1871

In [225]:
len(entity2longevity)

531

In [226]:
aaaa=0
for i in entity2aging.keys():
    if entity2longevity.get(i):
        aaaa+=1
print(aaaa)

373


In [227]:
for i in all_es:
    if entitydict.get(i.get('entity')):
        i['network']=True
    else:
        i['network']=False
    if entity2aging.get(i.get('entity')):
        i['aging']=True
    else:
        i['aging']=False
    if entity2longevity.get(i.get('entity')):
        i['longevity']=True
    else:
        i['longevity']=False

In [228]:
all_es[-1]

{'entity': 'Ototoxicity',
 'PMID': ['37319406'],
 'literature': 1,
 'allyears': [2023],
 'type': 'Disease',
 'MeshID': 'D000081015',
 'url': 'https://www.ncbi.nlm.nih.gov/mesh/2031054',
 'Summary': 'Damage to the EAR or its function secondary to exposure to toxic substances such as drugs used in CHEMOTHERAPY; IMMUNOTHERAPY; or RADIATION.',
 '_id': 'Disease_Ototoxicity',
 'datest': 20231101,
 'allwords': ['COCHLEOTOXICITY'],
 'network': True,
 'aging': False,
 'longevity': False}

In [229]:
json.dump(all_es,open('results/step4/all_es.json','w'))

In [230]:
len(all_es)

12315

In [231]:
entity_es_set=set()
for i in all_es:
    entity_es_set.add(i.get('entity'))

In [232]:
len(list(entity_es_set))

12257

In [233]:
names=locals()
for i in entity_pairs:
    names[i+'_all']=[]
    for j in all_es:
        if j.get('type')==i.replace('_',' '):
            names[i+'_all'].append(j)

In [234]:
len(Gene_all)

5624

In [235]:
len(Disease_all)

3501

In [236]:
Gene_all_sorted=sorted(Gene_all,key=lambda x:x['literature'],reverse=True)

printdata=[]
for i in entity_pairs:
    print('\n')
    print(i)
    names[i+'_all_sorted']=sorted(names[i+'_all'],key=lambda x:x['literature'],reverse=True)
    names[i+'_all_sorted_num']=0
    for j in names[i+'_all_sorted'][:10]:
        printdata.append({'y':10,'color':'colors[0]'})
        print("'"+j.get('entity')+"',")
for i in entity_pairs:
    print('\n')
    print(i)
    for j in names[i+'_all_sorted'][:10]:
        print(str(j.get('literature'))+",")
        names[i+'_all_sorted_num']+=j.get('literature')
for i in entity_pairs:
    print('\n')
    print(i)
    cxv=0
    for j in names[i+'_all_sorted'][:10]:
        z=j.get('literature')*100/names[i+'_all_sorted_num']
        print(str(round(z,2))+",")

In [238]:
len(RNA_all)

388

In [279]:
for i in entity_pairs:
    print('\n')
    print(i)
    names[i+'_all_sorted']=sorted(names[i+'_all'],key=lambda x:x['literature'],reverse=True)
    names[i+'_all_sorted_num']=0
    for j in names[i+'_all_sorted'][:10]:
        names[i+'_all_sorted_num']+=j.get('literature')
    print(names[i+'_all_sorted_num'])
    print('\n')
    for j in names[i+'_all_sorted'][:10]:
        print("'"+j.get('entity')+"',")
    print('\n')
    for j in names[i+'_all_sorted'][:10]:
        print(str(j.get('literature'))+",")
    print('\n')
    for j in names[i+'_all_sorted'][:10]:
        z=j.get('literature')*100/names[i+'_all_sorted_num']
        print(str(round(z,2))+",")
    print('\n')
# for i in entity_pairs:
#     print('\n')
#     print(i)
#     for j in names[i+'_all_sorted'][:10]:
#         print(str(j.get('literature'))+",")
#         names[i+'_all_sorted_num']+=j.get('literature')
# for i in entity_pairs:
#     print('\n')
#     print(i)
#     cxv=0
#     for j in names[i+'_all_sorted'][:10]:
#         z=j.get('literature')*100/names[i+'_all_sorted_num']
#         print(str(round(z,2))+",")



Gene
12616


'INS',
'CRP',
'IL6',
'APOE',
'CD4',
'MAPT',
'TNF',
'ALB',
'SIRT1',
'CD8A',


3014,
1830,
1375,
1363,
1244,
927,
893,
713,
680,
577,


23.89,
14.51,
10.9,
10.8,
9.86,
7.35,
7.08,
5.65,
5.39,
4.57,




RNA
324


'H19',
'MIR34A',
'TERC',
'MIR21',
'MIR146A',
'MIR155',
'MIR132',
'MIR221',
'HCCAT5',
'MIR31',


64,
60,
53,
42,
31,
18,
15,
14,
14,
13,


19.75,
18.52,
16.36,
12.96,
9.57,
5.56,
4.63,
4.32,
4.32,
4.01,




Protein
882


'Infliximab',
'Rituximab',
'Bevacizumab',
'Denosumab',
'Trastuzumab',
'Nivolumab',
'Cetuximab',
'Palivizumab',
'Ipilimumab',
'Ranibizumab',


314,
184,
96,
63,
61,
43,
37,
32,
28,
24,


35.6,
20.86,
10.88,
7.14,
6.92,
4.88,
4.2,
3.63,
3.17,
2.72,




Carbohydrate
6821


'Glucose',
'Ascorbic Acid',
'Hyaluronic Acid',
'Heparin',
'Doxorubicin',
'Sugars',
'Polysaccharides',
'Lipopolysaccharides',
'Digoxin',
'Fluorodeoxyglucose F18',


3900,
678,
401,
331,
298,
269,
263,
248,
225,
208,


57.18,
9.94,
5.88,
4.85,
4.37,
3.94,
3.86,
3.64,
3.3,
3.05,




Lip

In [ ]:
all_es_result=json.load(open('results/step2/all_es_result.json'))

In [ ]:
for i in entity_pairs:
    names[i.replace(' ','_')+'_sentences']=set()
for i in all_es_result:
    for j in i.get('sentence'):
        for k in j:
            if re.search('RNA',i.get('type')):
                names['RNA'.replace(' ','_')+'_sentences'].add(k)
            else:
                names[i.get('type').replace(' ','_')+'_sentences'].add(k)

In [ ]:
for i in entity_pairs:
    print(i)
    print(len(names[i.replace(' ','_')+'_sentences']))

Gene
93798
RNA
2341
Protein
2305
Carbohydrate
18383
Lipid
17388
Peptide
3850
Pharmaceutical_Preparations
297
Toxin
579
Mutation
4147
Disease
758812


# Entity_Info.json

In [248]:
all_es_result_dict={}
for i in all_es_result:
    if not all_es_result_dict.get(i.get('entity')):
        all_es_result_dict[i.get('entity')]=i

In [249]:
all_es_result[10000]

{'entity': 'Keratosis, Seborrheic',
 'PMID': ['34830382',
  '32646224',
  '31722568',
  '31508199',
  '30954583',
  '29168986',
  '27593207',
  '25798694',
  '23427523',
  '23331299',
  '21054564',
  '18688106',
  '11493097',
  '8682971',
  '8496484',
  '1621628',
  '1533233',
  '2566238',
  '36065721',
  '36932111',
  '36610814',
  '37467376',
  '37843094'],
 'target': [['Seborrheic keratosis',
   'seborrheic keratosis',
   'Seborrheic keratosis',
   'seborrheic keratosis',
   'seborrheic keratosis'],
  ['seborrheic keratosis', 'seborrheic keratosis'],
  ['Seborrheic keratosis', 'SK'],
  ['Seborrheic keratosis', 'SK', 'SK', 'SK'],
  ['seborrheic keratosis'],
  ['seborrheic keratosis', 'Seborrheic keratosis'],
  ['seborrheic keratosis', 'seborrheic keratoses', 'seborrheic keratosis'],
  ['Seborrheic keratoses', 'SK', 'SK'],
  ['Seborrheic keratosis', 'seborrheic keratosis'],
  ['seborrheic keratosis'],
  ['seborrheic keratosis'],
  ['seborrheic keratosis'],
  ['Seborrheic keratosis'],


In [250]:
all_es_dict={}
for i in all_es:
    if not i.get('url'):
        i['url']=i.get('NCBI_URL')
    j={'entity':i.get('entity'),'type':i.get('type'),'PMID':i.get('PMID'),
       'official full name':i.get('Official_Full_Name'),
       'sentence':all_es_result_dict.get(i.get('entity')).get('sentence'),
      'numbers of articles':i.get('literature'),
       'JT':all_es_result_dict.get(i.get('entity')).get('JT'),
       'TA':all_es_result_dict.get(i.get('entity')).get('TA'),
       'IF':all_es_result_dict.get(i.get('entity')).get('IF'),
       'IF5':all_es_result_dict.get(i.get('entity')).get('IF5'),
       'year':all_es_result_dict.get(i.get('entity')).get('year'),
       'date':all_es_result_dict.get(i.get('entity')).get('date'),
       'alias names':i.get('Also_known_as',''),
       'description':i.get('Summary',''),
       'url':i.get('url',''),
       'mutation position':i.get('position',''),
       'mutation alleles':i.get('Alleles',''),
       'MeSH ID':i.get('MeshID',''),
       'relation':i.get('network'),
       'external links':[],
       'aging biomarker':i.get('aging'),
       'longevity biomarker':i.get('longevity')}
    if i.get('NCBI_URL'):
        j['external links']+=[{'NCBI':{'NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID')}}]
    if i.get('GenAge_URL'):
        j['external links']+=[{'GenAge':{'GenAge_URL':i.get('GenAge_URL'),'GenAge_symbol':i.get('GenAge_symbol')}}]
    if i.get('LongevityMap_URL'):
        j['external links']+=[{'LongevityMap':{'LongevityMap_URL':i.get('LongevityMap_URL'),'LongevityMap_symbol':i.get('LongevityMap_symbol')}}]
    if i.get('AgingBank_URL'):
        j['external links']+=[{'AgingBank':{'AgingBank_URL':i.get('AgingBank_URL'),'AgingBank_symbol':i.get('AgingBank_symbol')}}]
    if i.get('AgingAtlas_URL'):
        j['external links']+=[{'AgingAtlas':{'AgingAtlas_URL':i.get('AgingAtlas_URL'),'AgingAtlas_symbol':i.get('AgingAtlas_symbol')}}]
    if not all_es_dict.get(i.get('entity')):
        all_es_dict[i.get('entity')]=[j]
    else:
        all_es_dict[i.get('entity')]=all_es_dict.get(i.get('entity'))+[j]

In [251]:
len(all_es_dict)

12257

In [252]:
all_es_length=set()
for i,j in all_es_dict.items():
    for k in j[0].get('PMID'):
        all_es_length.add(k)
print(len(all_es_length))

183893


In [253]:
json.dump(all_es_dict,open('results/step5/Entity_Info.json','w'))

In [254]:
list(all_es_dict.items())[0]

('MLH1',
 [{'entity': 'MLH1',
   'type': 'Gene',
   'PMID': ['12612901',
    '30275527',
    '25311944',
    '22936446',
    '19949675',
    '22406557',
    '23240038',
    '11325821',
    '21042749',
    '25556597',
    '17556535',
    '29425284',
    '22740444',
    '10954253',
    '37380216'],
   'official full name': 'mutL homolog 1',
   'sentence': [['Most such cancers have the CpG island methylator phenotype (CIMP+) with methylation and transcriptional silencing of the mismatch repair gene MLH1.'],
    ['Our group recently demonstrated that aging human HSCs accumulate microsatellite instability coincident with loss of MLH1, a DNA Mismatch Repair (MMR) protein, which could reasonably predispose to radiation-induced HSC malignancies.',
     'In addition, whole-exome sequencing analysis revealed high SNVs and INDELs in lymphomas being driven by loss of Mlh1 and frequently mutated genes had a strong correlation with human leukemias.'],
    ['ARID1A loss was observed in 9% (22/257) of

# Relation_Info.json

In [256]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [257]:
Relation_Info_list=[]
Relation_Info_dict={}
for i in triplelistnew:
    if i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed') not in Relation_Info_dict.keys():
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]={
            'source entity':i.get('entity_one_transformed'),
            'relationship':i.get('triple')[1],
            'target entity':i.get('entity_two_transformed'),
            'sentence':[i.get('sentence')],
            'source':[i.get('entity_one')],
            'target':[i.get('entity_two')],
            'source type':entitydict_tolist.get(i.get('entity_one_transformed')),
            'target type':entitydict_tolist.get(i.get('entity_two_transformed')),
            'PMID':[i.get('PMID')],
            'DP':[pubtator_info_dict.get(i.get('PMID')).get('DP')],
            'date':[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))],
            'TI':[pubtator_info_dict.get(i.get('PMID')).get('TI')],
            'TA':[pubtator_info_dict.get(i.get('PMID')).get('TA')],
            'IF':[i.get('IF')],
            'IF5':[i.get('IF5')],
            'method':i.get('method')}
    else:
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['PMID']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('PMID')+[i.get('PMID')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['sentence']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('sentence')+[i.get('sentence')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['source']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('source')+[i.get('entity_one')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['target']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('target')+[i.get('entity_two')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['DP']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('DP')+[pubtator_info_dict.get(i.get('PMID')).get('DP')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['TI']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('TI')+[pubtator_info_dict.get(i.get('PMID')).get('TI')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['TA']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('TA')+[pubtator_info_dict.get(i.get('PMID')).get('TA')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['date']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('date')+[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['IF']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('IF')+[i.get('IF')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['IF5']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('IF5')+[i.get('IF5')]
        if i.get('method')[0] not in Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('method'):
            Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['method']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('method')+i.get('method')
        else:
            continue
            

In [258]:
list(Relation_Info_dict.items())[10]

('Sarcopenia-associated-Constipation',
 {'source entity': 'Sarcopenia',
  'relationship': 'associated',
  'target entity': 'Constipation',
  'sentence': ['(4) Conclusions: Sarcopenia and slow gait speed associated with constipation in community-dwelling older adults.'],
  'source': ['Sarcopenia'],
  'target': ['constipation'],
  'source type': ['Disease'],
  'target type': ['Disease'],
  'PMID': ['34769606'],
  'DP': ['2021 Oct 21'],
  'date': [20211021],
  'TI': ['Status of Constipation and Its Association with Sarcopenia in Older Adults: A Population-Based Cohort Study.'],
  'TA': ['Int J Environ Res Public Health'],
  'IF': [0.0],
  'IF5': [0.0],
  'method': ['open domain']})

In [259]:
len(Relation_Info_dict)

116495

In [260]:
Relation_Info_length=set()
for i,j in Relation_Info_dict.items():
    for k in j.get('PMID'):
        Relation_Info_length.add(k)
print(len(Relation_Info_length))

50696


In [261]:
json.dump(Relation_Info_dict,open('results/step5/Relation_Info.json','w'))

# Aging_Biomarkers.json

In [262]:
len(positive_relations)

8416

In [263]:
positive_relations[0]

{'triple': ['alanine aminotransferase', 'recognized', 'mortality'],
 'PMID': '34063029',
 'sentence': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty.',
 'AB': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty. In the elderly, both frailty and ischemic stroke are not only common, but are also associated with mortality. The aim of this research was to investigate whether a significantly reduced ALT level increases the all-cause mortality rate in the elderly with ischemic stroke. (2) Methods: Between February 2014 and April 2019, a retrospective study of 901 patients with ischemic stroke admitted to a university-affiliated hospital was conducted. Cox proportional hazard regression was used to determine whether a significantly reduced AL

In [264]:
pubtator_info_dict_TI={}
pubtator_info_dict_DP={}
for i in pubtator_info:
    pubtator_info_dict_TI[i.get('PMID')]=i.get('TI')
for i in pubtator_info:
    pubtator_info_dict_DP[i.get('PMID')]=i.get('DP')

In [265]:
Aging_Biomarkers_dict={}
for i in positive_relations:
    if not Aging_Biomarkers_dict.get('entity_one_transformed'):
        Aging_Biomarkers_dict[i.get('entity_one_transformed')]=[{
        'source entity':i.get('entity_one_transformed'),
            'relationship':i.get('triple')[1],
            'target entity':i.get('entity_two_transformed'),
            'sentence':i.get('sentence'),
            'source':i.get('entity_one'),
            'target':i.get('entity_two'),
            'source type':i.get('entity_one_type'),
            'target type':i.get('entity_two_type'),
            'PMID':i.get('PMID'),
            'DP':pubtator_info_dict_DP.get(i.get('PMID')),
            'date':i.get('date'),
            'TI':pubtator_info_dict_TI.get(i.get('PMID')),
            'TA':i.get('TA'),
            'IF':i.get('IF'),
            'IF5':i.get('IF5')
        }]

In [266]:
list(Aging_Biomarkers_dict.items())[0]

('GPT',
 [{'source entity': 'GPT',
   'relationship': 'correlated',
   'target entity': 'Death',
   'sentence': 'RESULTS: Profiling of blood parameters demonstrated that elevated levels of alanine aminotransferase (ALT), total bilirubin (T-bil), blood urea nitrogen (BUN), creatinine (Cr) and a decreased platelet count were significantly correlated with death within 1 week in a training cohort.',
   'source': 'alanine aminotransferase',
   'target': 'death',
   'source type': 'Gene',
   'target type': 'Disease',
   'PMID': '28011502',
   'DP': '2017 Jan',
   'date': 20170101,
   'TI': 'Objective Predictive Score as a Feasible Biomarker for Short-term Survival in TerminalIy Ill Patients with Cancer.',
   'TA': 'Anticancer Res',
   'IF': '2.0',
   'IF5': '2.2'}])

In [267]:
len(Aging_Biomarkers_dict)

1871

In [268]:
Aging_Biomarkers_length=set()
for i,j in Aging_Biomarkers_dict.items():
    for k in j:
        Aging_Biomarkers_length.add(k.get('PMID'))
print(len(Aging_Biomarkers_length))

1507


In [269]:
json.dump(Aging_Biomarkers_dict,open('results/step5/Aging_Biomarkers.json','w'))

# Longevity_Biomarkers.json

In [270]:
Longevity_Biomarkers_dict={}
for i in negative_relations:
    if not Longevity_Biomarkers_dict.get('entity_one_transformed'):
        Longevity_Biomarkers_dict[i.get('entity_one_transformed')]=[{
        'source entity':i.get('entity_one_transformed'),
            'relationship':i.get('triple')[1],
            'target entity':i.get('entity_two_transformed'),
            'sentence':i.get('sentence'),
            'source':i.get('entity_one'),
            'target':i.get('entity_two'),
            'source type':i.get('entity_one_type'),
            'target type':i.get('entity_two_type'),
            'PMID':i.get('PMID'),
            'DP':pubtator_info_dict_DP.get(i.get('PMID')),
            'date':i.get('date'),
            'TI':pubtator_info_dict_TI.get(i.get('PMID')),
            'TA':i.get('TA'),
            'IF':i.get('IF'),
            'IF5':i.get('IF5')
        }]

In [271]:
list(Longevity_Biomarkers_dict.items())[0]

('Glucose',
 [{'source entity': 'Glucose',
   'relationship': 'attenuate',
   'target entity': 'Cerebrovascular Disorders',
   'sentence': 'Raising NAD+ levels in model organisms by administration of NAD+ precursors improves glucose and lipid metabolism; attenuates diet-induced weight-gain, diabetes, diabetic kidney disease, and hepatic steatosis; reduces endothelial dysfunction; protects heart from ischemic injury; improves left ventricular function in models of heart failure; attenuates cerebrovascular and neurodegenerative disorders; and increases health-span.',
   'source': 'glucose',
   'target': 'cerebrovascular and neurodegenerative disorders',
   'source type': 'Carbohydrate',
   'target type': 'Disease',
   'PMID': '37364580',
   'DP': '2023 Nov 9',
   'date': 20231109,
   'TI': 'Nicotinamide Adenine Dinucleotide in Aging Biology: Potential Applications and Many Unknowns.',
   'TA': 'Endocr Rev',
   'IF': '20.3',
   'IF5': '25.8'}])

In [272]:
len(Longevity_Biomarkers_dict)

531

In [273]:
Longevity_Biomarkers_length=set()
for i,j in Longevity_Biomarkers_dict.items():
    for k in j:
        Longevity_Biomarkers_length.add(k.get('PMID'))
print(len(Longevity_Biomarkers_length))

500


In [274]:
json.dump(Longevity_Biomarkers_dict,open('results/step5/Longevity_Biomarkers.json','w'))

## Evaluation

In [275]:
num_of_aging=set()
num_of_longevity=set()
for i in disease_list:
    if i.get('partofspeech')=='positive':
        num_of_aging.add(i.get('entity_one_transformed'))
    elif i.get('partofspeech')=='negative':
        num_of_longevity.add(i.get('entity_one_transformed'))
print(len(num_of_aging))
print(len(num_of_longevity))

1871
531


In [276]:
all_es_count=set()
print(len(all_es))
for i in all_es:
    all_es_count.add(i.get('entity'))
print(len(all_es_count))

12315
12257


In [277]:
print(len(relation_interpolation_list))

116495


In [280]:
collections_pubmed_raw_pmid=db.pubmed_raw_pmid
len(list(collections_pubmed_raw_pmid.find()))

342651